To run this notebook is it required an Api Key from OpenAI get one in https_//platform.openai.com/api-keys.

The model used is:
- gpt-5-nano


### Tutorial LangChain but updated to use OpenAI from https://docs.langchain.com/oss/python/langchain/quickstart

### Test OpenAI connection

In [7]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

MODEL='gpt-5-nano'

load_dotenv()  # Loads .env variables as environment variables

model = init_chat_model(MODEL)
response = model.invoke("Why do parrots talk? in 5 words")
print(response.content)

They imitate sounds they've heard.


In [5]:
# Basic streaming response
for chunk in model.stream("Why do parrots have colorful feathers? in 5 words"):
    print(chunk.text, end="|", flush=True)

|To| attract| mates| and| communicate|.||||

### Build a basic agent OpenAI

In [10]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=MODEL,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='d31705fa-5fe4-4f80-968c-ca606b8b7254'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 216, 'prompt_tokens': 140, 'total_tokens': 356, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CmogHBrHIIVhuCIGcxbD9biRuE8ek', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b1eef-e666-7950-be88-bf81cad67229-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_wOlJzsg9qYKWbgZ4KQ0XTQ2X', 'type': 'tool_call'}], usage_metadata={'input_tokens': 140, 'output_tokens': 216, 'total_tok

In [19]:
# https://docs.langchain.com/oss/python/integrations/chat/ollama
from typing import List

from langchain.messages import AIMessage
from langchain.tools import tool
from langchain_openai import ChatOpenAI

@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm = ChatOpenAI(model=MODEL,
                 temperature=0,
                ).bind_tools([validate_user])

messages = [("system", "you are a helpful assistant"),
            ("human", "Could you validate user 123? They previously lived at 123 Fake St in Boston MA and 234 Pretend Boulevard in Houston TX."),
]

result = llm.invoke(messages)

if isinstance(result, AIMessage) and result.tool_calls:
    print(result.tool_calls)

[{'name': 'validate_user', 'args': {'user_id': 123, 'addresses': ['123 Fake St in Boston MA', '234 Pretend Boulevard in Houston TX']}, 'id': 'call_FidRPhBMnDrmzJikMClzsfc8', 'type': 'tool_call'}]


In [20]:
# Using a non reasoning model
from langchain.messages import HumanMessage
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=MODEL)

messages = [
    HumanMessage("What is 3^3?"),
]

response = llm.invoke(messages)
print(response.content)

27

(3 cubed = 3 × 3 × 3 = 27)


### Build a real-world agent

This example is available in https://docs.langchain.com/oss/python/langchain/quickstart using Claude Sonnet 4.5.
For this example I used OpenAI an local model 

In [22]:
# Define the system prompt
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.
            You have access to two tools:
            - get_weather_for_location: use this to get the weather for a specific location
            - get_user_location: use this to get the user's location
            If a user asks you for the weather, make sure you know the location. If you can tell 
            from the question that they mean wherever they are, use the get_user_location tool 
            to find their location."""

In [23]:
# Create tools
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    print(f'get_weather_argument : {city}')
    if city == "Florida":
        return f"It's always sunny in {city}!"
    return f"It's raining in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    print(f'get_user_location_argument : {user_id}')
    return "Florida" if user_id == "1" else "SF"

In [24]:
# Configure your model
from langchain.chat_models import init_chat_model

model=init_chat_model(model=MODEL, 
                      temperature=0.5,
                      )


In [25]:
# Add memory
from langgraph.checkpoint.memory import InMemorySaver
checkpointer = InMemorySaver()

In [26]:
def responseFormat(response):
    for item in response["messages"]:
        if isinstance(item, HumanMessage):
            if item.content:
                print(f"Query : {item.content}")
        if isinstance(item, AIMessage):
            if item.content:
                print(f"AI response: {item.content}")

In [27]:
# Create and run model
from langchain.messages import AIMessage
from langchain.messages import HumanMessage

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    checkpointer=checkpointer
)

In [28]:
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me my location and what the weather outside is."}]},
    config=config,
    context=Context(user_id="1")
)

# # Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

responseFormat(response)

get_user_location_argument : 1
get_weather_argument : Florida
Query : Tell me my location and what the weather outside is.
AI response: Location: Florida
Weather: It’s always sunny in Florida!

Punny tip: Sun’s out, fun’s out—slap on some sunscreen and grab those shades before you soak up that radiant day.

Want a city-specific or hourly forecast for where you are in Florida? I can drizzle you with more details.
Query : thank you!
AI response: You're welcome! Happy to keep the sunshine on your side. If you want more detail—city-level, hourly, rain chances, or UV index—just say the word and I’ll give you the full forecast. Until then, stay sunny and sunscreen-up!


In [29]:
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "2"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me my location and what the weather outside is."}]},
    config=config,
    context=Context(user_id="2")
)

# # Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="2")
)

responseFormat(response)

get_user_location_argument : 2
get_weather_argument : San Francisco
Query : Tell me my location and what the weather outside is.
AI response: You're in San Francisco. Weather: It's raining in San Francisco.

Tip: grab an umbrella or a rain jacket—the city by the bay is putting on a wet show. Stay dry out there!
Query : thank you!
AI response: You're welcome! If you want another forecast—or just a sunny pun—I'm here to weather the request. Have a great day!
